In [6]:
from src.evaluate import Evaluate
from src.supply_chain_config import SupplyChainConfig
from src.policy import MeanDemandPolicy, StaticBaseStockPolicyRandom, StaticBaseStockPolicyShortfall
from src.dynamics import Dynamics

import numpy as np
import pandas as pd

In [49]:
def create_deviation(deviations):
    return [np.random.randint(0, deviation) for deviation in deviations]

<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Initialize network & a base strategy
</div>

In [50]:
# Create an instance of supply chain configuration
supply_chain_config = SupplyChainConfig()
dynamics = Dynamics(supply_chain_config)

# Policy parameters
hedging_base_stock_shortfall = 0.25
hedging_base_stock_random = 0.25
hedging_mean_demand_policy = 0.1

# Creat a base policy
base_stock_policy_random = StaticBaseStockPolicyRandom(supply_chain_config, hedging_base_stock_random)
base_stock_levels = np.array(base_stock_policy_random.base_stock_levels)

<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Generate the data
</div>

In [58]:
# Parameters for testing
num_policies = 1000
num_demand_patterns = 100
num_trajectories = 1
periods_per_trajectory = 200
evaluate = Evaluate(supply_chain_config, dynamics, num_trajectories, periods_per_trajectory)

policies = [base_stock_policy_random]
for n in range(num_policies - 1):
    policies.append(StaticBaseStockPolicyRandom(supply_chain_config, hedging_base_stock_random, deviations = create_deviation(base_stock_levels // 10)))

data_parts = list()
for demand_pattern in range(num_demand_patterns):
    costs = evaluate.compare_policies(policies)
    data_parts.append(np.concatenate([np.arange(0, num_policies).reshape(-1, 1),
                                                np.ones((num_policies, 1)) * demand_pattern, 
                                                np.array([policy.base_stock_levels for policy in policies]), 
                                                np.array(costs).reshape(-1, 1)], axis = 1))

df = pd.DataFrame(np.concatenate(data_parts, axis = 0), columns = ['policy_id', 'demand_pattern'] + [f"C{i+1}" for i in range(len(base_stock_policy_random.base_stock_levels))] + ['cost'])
df.to_csv('data/generated_data.csv', index = False)

KeyboardInterrupt: 